In [44]:
import pandas
import trueskill
from collections import OrderedDict
import scipy.stats

In [54]:
df = pandas.read_csv('dump.csv')

In [55]:
def learn_trueskill_ratings(df):
    methods = df['name'].unique()
    ratings = {method: trueskill.Rating(mu=0, sigma=0.5) for method in methods}

    for _, group in df.groupby(['example_id', 'username']):
        ranks = []
        players = []
        for row in group.itertuples():
            players.append(row.name)	
            ranks.append(row.score - 1)

        player_ratings = [(ratings[player],) for player in players]
        
        new_ratings = trueskill.rate(player_ratings, ranks=ranks)
        for player, (rating,) in zip(players, new_ratings):
            ratings[player] = rating
    
    return ratings

ratings = learn_trueskill_ratings(df)
print(ratings)



{'attention_all_layers': trueskill.Rating(mu=1.706, sigma=0.662), 'int_gradients': trueskill.Rating(mu=-1.714, sigma=0.664), 'attention_learned': trueskill.Rating(mu=4.433, sigma=0.688), 'gradient_x_input': trueskill.Rating(mu=-4.429, sigma=0.689)}


In [56]:
def partial_ordered_ranks(ratings):
    intervals = {}
    for method, rating in ratings.items():
        intervals[method] = scipy.stats.norm.interval(0.95, rating.mu, rating.sigma)

    sorted_intervals = sorted(intervals.items(), key=lambda x: x[1][1], reverse=True)
    current_rank = 0
    prev_start = float('+inf')
    ranks = {}
    for method, interval in sorted_intervals:
        if interval[1] < prev_start:
            current_rank += 1

        ranks[method] = current_rank
        prev_start = min(prev_start, interval[0])
        
    return ranks

partial_ranks = partial_ordered_ranks(ratings)
print(partial_ranks)

{'attention_learned': 1, 'attention_all_layers': 2, 'int_gradients': 3, 'gradient_x_input': 4}


In [8]:



systems = ["a", "b", "c"]
rankings = OrderedDict()
for system in systems:
    rankings[system] = trueskill.Rating(mu=0, sigma=0.5)

[(trueskill.Rating(mu=0.045, sigma=0.505),),
 (trueskill.Rating(mu=-0.022, sigma=0.505),),
 (trueskill.Rating(mu=-0.022, sigma=0.505),)]